# KLT Tracker mit Weights

## Theorie

### Variablen

- $T$ für Template, Das Tamplate versucht der Tracker zu Matchen
- $I$ für Image, ist das Bild in dem das Template T gesucht wird
- $N_0$ ist die Menge aller Pixel im Tamplate
- $x \in N_0$ ist die Zählvariable, die die Pixel durchläuft 
- $p$ ist die aktuelle Verschiebung von der Ursprünglichen position des Templates
- $\Delta p$ ist die Differenz zwischen dem nächsten p und dem aktuellem p
- $c$ für color, ist die Laufvariable, die die Frabkanäle druchläuft

### Ziel:

Minimierung der folgenden Funktion: $E$

\begin{equation}
E(\Delta p)=\sum_{x} w_c w_d |I(x+p+\Delta p)-T(x)|^2
\end{equation}

wobei: 
\begin{equation}
w_d = exp \left( -\frac{|x_0-x|^2}{2\delta_d} \right)
\end{equation}
\begin{equation}
w_c(x) = exp \left( -\frac{|T(x_0)-T(x)|^2}{2\delta_c} \right) 
\end{equation}


### Umstellen nach $\Delta p$


\begin{equation}
E(\Delta p)=\sum_{x}w_c w_d \sum_{c}  |I_c(x+p+\Delta p)-T_c(x)|^2
\end{equation}
\begin{equation}
    \approx \sum_{x}w_c w_d \sum_{c} |I_c(x+p)+\nabla I_c(x+p)^T \Delta p -T_c(x)|^2
\end{equation}

Abgeleitet:
\begin{equation}
\sum_{x}w_c w_d \sum_{c} \nabla I_c(x+p) \left(I_c(x+p)+\nabla I_c(x+p)^T \Delta p -T_c(x)\right) = 0
\end{equation}

\begin{equation}
\implies -\sum_{x}w_c w_d \sum_{c} \nabla I_c(x+p) \left(I_c(x+p)-T_c(x) \right) = \sum_{x}w_c w_d \sum_{c} \nabla I_c(x+p)\nabla I_c(x+p)^T \Delta p 
\end{equation}

Definiere:

\begin{equation}
H =  \sum_{x}w_c w_d \sum_{c} \nabla I_c(x+p) \nabla I_c(x+p)^T
= \sum_{c} \sum_{x}w_c w_d  \nabla I_c(x+p) \nabla I_c(x+p)^T
\end{equation}


\begin{equation}
Z = \sum_{c} \sum_{x}w_c w_d  \nabla I_c(x+p) \left(I_c(x+p)-T_c(x) \right)
\end{equation}

Es folgt:

\begin{equation}
\Delta p = -H^{-1} \sum_{c} \sum_{x}w_c w_d  \nabla I_c(x+p) \left(I_c(x+p)-T_c(x) \right) = -H^{-1}Z
\end{equation}

Man kann H und Z gleichzeitig berechnen:
\begin{equation}
\begin{pmatrix} Z \\ H \end{pmatrix} 
= \sum_{c} \sum_{x}w_c w_d \nabla I_c(x+p)
\begin{pmatrix}
 \left(I_c(x+p)-T_c(x) \right) \\ 
 \nabla I_c(x+p)^T 
\end{pmatrix}
= \sum_{c} \sum_{x}w_c w_d 
\begin{pmatrix}
 \nabla I_c(x+p) \left(I_c(x+p)-T_c(x) \right) \\ 
 \nabla I_c(x+p) \nabla I_c(x+p)^T 
\end{pmatrix}
\end{equation}



# Algorithmus

Input: $N_0$, $T$, $I$, $\triangledown T$, $\epsilon$, $\delta_c$, $\delta_d$

Output: $p$

Legen $w_c w_d$ zum Speichern an



1. $p \gets \begin{bmatrix}
           0 \\
           0 
         \end{bmatrix}$
         
2.  $\Delta p \gets \begin{bmatrix}
           \epsilon \\
           \epsilon
         \end{bmatrix}$

3. $T_{c,x}$ bekommen

4. Berechne:
$
w_c w_d =  exp \left( -\frac{|x_0-x|^2}{2\delta_d } -\frac{|T(x_0)-T(x)|^2}{2\delta_c}\right)
$

5. Solange $|\Delta p|>\epsilon$
    1. $H \gets \begin{bmatrix}
           0 & 0\\
           0 & 0
         \end{bmatrix}$ 

    2. $ Z \gets \begin{bmatrix}
           0 \\
           0 
         \end{bmatrix}$
    3. $\tilde x = x+p$

    4. Iteriere über c:

        1. Bekomme $\nabla I_c(\tilde x)$

        2. Berechne: $e_c = \left(I_c(\tilde x)-T_c(x) \right)$

        3. Berechne $z_c = \nabla I_c(\tilde x) e_c$

        4. Berechne $Z_c = \sum_{x}w_c w_d z_c $

        5. $Z \gets Z + Z_c$

        6. Berechne: $h_c = \nabla I_c(\tilde x) \nabla I_c(\tilde x)^T$ 

        7. $ H_c = \sum_{x}w_c w_d h_c $

        8. $H \gets H + H_c$ 

    4. Berechne $H^{-1}$

    5. $ \Delta p = -H^{-1}Z $
    
    6. $p \gets \Delta p + p$




# Implementierung

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Weights berechnen

benutzte Formel: 
$
w_c w_d =  exp \left( -\frac{|x_0-x|^2}{2\delta_d } -\frac{|T(x_0)-T(x)|^2}{2\delta_c}\right)
$

Schritte:

1. $distance = x_0-x$
2. $normDistance = |x_0-x|^2$
3. $colorDiff = T(x_0)-T(x)$
4. $normColor = |T(x_0)-T(x)|^2 = \sum_c \left( T_c(x_0)-T_c(x) \right)^2$
5. $exponentDistance = -\frac{normDistance}{2\delta_d}$
6. $exponentColor = -\frac{normColor}{2\delta_c}$
6. $weights = exp(exponentDistance+exponentColor)$

In [ ]:
def computeWeights(x1,x2,T_x,delta_c,delta_d,x_0,Tx_0):
    #Schritt 1
    distanceX1 = x1-x_0[0]
    distanceX2 = x2-x_0[1]
    #Schritt2
    normDistance = distanceX1**2 + distanceX2**2
    
    normColor = None
    cDim = T_x.shape[0]
    
    for c in range(cDim):
        #Schritt 3
        colorDiff = T_x[c]-Tx_0[c]
        #Schritt 4        
        if(normColor is None):
            normColor = colorDiff**2
        else:
            normColor += colorDiff**2
    
    #Schritt 5
    exponentDistance = - normDistance/(2*delta_d)
    
    #Schritt 6
    exponentColor = -normColor/(2*delta_c)
    
    #Schritt 7
    weights = np.exp(exponentColor+exponentDistance)
    
    return weights
    

# H berechnen

Eigentlich berechnet die Funktion in der Hauptfunktion nur $h_c$
Jedoch kann man die Funktion auch für ganz H benutzten, wenn man alle gradients hinzugibt

Die benutzte Formel: $H = \sum_{x} w_c w_d  \triangledown T(x) {\triangledown T(x)}^T$

In [ ]:
def compute_h(gradx1,gradx2,weights):
    # gradx1 und gradx2 solltem 1D sein
    #Schritt f
    h = np.array([[gradx1**2,gradx2*gradx1],
                  [gradx2*gradx1,gradx2**2]])
    #Schritt g
    H=h*weights
    
    return H.sum(axis=2)
    

# Z berechen

Die Benutzen Formeln

5.4.b. $e = I(\tilde{x})-T(x)$

5.4.c. $z = {\triangledown T(x)} e$

5.4.d. $Z = \sum_x w_c w_d z$

In [ ]:
def computeZ(I,T,gradIX1,gradIX2,weigts):
    #Schritt b
    e = I-T
    #Schritt c
    gradX1 = gradIX1*e
    gradX2 = gradIX2*e
    #Schritt d
    ZX1 = weights * gradX1
    ZX2 = weights * gradX2
    
    return np.array([ZX1.sum(),ZX2.sum()])    

### Hauptfunktion

In [1]:
def trackTamplate(I,T,gradI,N_0L,N_0U,x_0,delta_c,delta_d,epsilon,maxIteration):
    """
    I,T sind numpy arrays der Form (color Dimension) des Typen interpolte2d
    gradI ist ein numpy array der Form (color Dimension) des Typen interpolte2d
    N_0L und N_0U sollen einen Viereck aufspannen. Wobei N_0L der untere (lower) Punkt ist,
    und N_0U der Obere (upper)
    x_0 ist der betrachtete Mittelpunkt und eine 2D List
    delta_c ist eine Zahl, die den Color weight bestimmt
    delta_d ist eine Zahl, die den Distance weight bestimmt
    epislon Wert ab dem die Schrittweite klein genug ist um abgebrochen zu werden
    maxIteration ist die maximale Anzahl der Iteration die der Algorithmus machen darf
    
    die Interpolate2d Klasse sollte so Funktionieren,
    dass f([1,2,3,4],[1,2,3]) die pixel an den Punken:
    [
    [[1,1],[1,2],[1,3]]
    [[2,1],[2,2],[2,3]]
    [[3,1],[3,2],[3,3]]
    [[4,1],[4,2],[4,3]]
    ]
    zurück gibt   

    """
    
    #Sicherheit
    if(len(I)!=len(T) or len(I)!=len(gradT)):
        raise Exception("I,T,gradT haben unterschiedliche Dimensionen")
    
    #Schirtt 1 & 2 
    colorDim = len(I)
    p = np.array([0,0])
    deltap = np.array([epsilon,epislon])

    x1Range = np.arange(N_0L[0],N_0U[0])    
    x2Range = np.arange(N_0L[1],N_0U[1])
    x1Dim = len(x1Range)
    x2Dim = len(x2Range)
    
    iterator = 0
    
    #Schritt 3
    T_x = np.empty(shape=(colorDim,x1Dim,x2Dim),dtype = np.float64)
    T_x0 = np.empty(shape=(colorDim),dtype = np.float64)
    
    for c in range(colorDim):
        T_x[c] = T[c](x1Range,x2Range)
        T_x0[c] = T[c](x_0[0],x_0[1])
        
    #Schritt 4
    X1,X2 = np.meshgrid(x1Range,x2Range)
    weights = computeWeights(X1,X2,T_x,delta_c,delta_d,x_0,T_x0)
    weightsForH = weights.flatten()
    weightsForH.shape=(xDim*yDim,1,1)
    weightsForH = weightsForH[:,[0,0]][:,:,[0,0]]
    
    
    #Schritt 5
    while(np.linalg.norm(deltap)>epsilon and iterator<maxIteration):
        
        #Schritt A
        H = np.array([[0,0],[0,0]])
        
        #Schritt B
        Z = np.array([0,0])
        
        #Schritt C
        x1Tilde = x1Range+p[0]
        x2Tilde = x2Range+p[1]
        
        #Schritt D
        for c in range(cDim):
            #Schritt a
            gradIX1 = gradI[c][0](x1Tilde,x2Tilde)
            gradIX2 = gradI[c][1](x1Tilde,x2Tilde)
            
            #Schritt b - d
            I_x = I[c](x1Tilde,x2Tilde)
            Z_c = computeZ(I_x,T_x[c],gradIX1,gradIX2,weights)
            
            #Schritt e
            Z+=Z_c
            
            #Schritt f-g
            H_c = computeH(gradIX1.flatten(),gradIX2.flatten(),weightsForH)
            
            #Schritt h
            H+=H_c
            
        #Schritt E
        Hinv = np.linalg.inv(H)
        
        #Schritt F
        deltap = -Hinv.dot(Z)
        
        #Schritt G
        p +=deltap
        
        iterator+=1
        
return p
            
        
    